# U-Net On Vaihingen Dataset

In this lab session, you will use a U-Net model to perform semantic segmentation on the Vaihingen Dataset.
The full Vaihingen dataset is available for download [here](https://www.kaggle.com/datasets/naydex/vaihingen-cropped), but in this lab you will use a smaller version (see details below) to have affordable training times.

As a reminder, the U-Net architecture is defined as follows :

[<img src="https://camo.githubusercontent.com/41ded1456b9dbe13b8d73d8da539dac95cb8aa721ebe5fb798af732ca9f04c92/68747470733a2f2f692e696d6775722e636f6d2f6a6544567071462e706e67" alt="drawing" width="1000">](https://github.com/milesial/Pytorch-UNet)

This notebook is derived from a nice U-Net implementation in PyTorch (click the image to access it). For more information, you can take a look at the [paper](https://arxiv.org/pdf/1505.04597.pdf)!

## The UNet Model

You will start by defining the UNet architecture to be used for this session. As a helper, the following blocks are defined:

* `double_conv2d`: each sequence of two consecutive convolutions in the UNet model can be replaced by such a block
* `down_sampling_block`: each sequence of a down-sampling followed by a two convolutions can be replaced by such a block
* `up_sampling_block`: each sequence of an up-sampling (transposed convolution) followed by two convolutions can be replaced by such a block.

**Question 1.** Using the Keras Functional API, define the UNet architecture shown above.

## The data

You will now download the mini-Vaihingen dataset from CURSUS and put the zip file in your working folder.
The following command should unzip it:

Now the following code defines a loader for the training data and another one for the validation data:

**Question 2.** Inspect the first batch (`train_loader[0]`) from the training set. What are the shapes of `X` and `y`? And what values can they take?

**Question 3.** Define, compile and fit (for one or two epochs, no more) a UNet model using the `unet_model` function defined above.
What accuracy can you reach on the validation set?

**Question 4.** Download the pre-trained model from CURSUS and put it in your working folder.
Load its weights using `model.load_weights("pretrained_unet.h5")` and evaluate its performance on the validation set using [`model.evaluate`](https://keras.io/api/models/model_training_apis/#evaluate-method).

## Let's see the results !

**Question 5.** Use the code below to visualize the obtained results on some of the validation samples.